In [1]:
import pandas as pd
import datetime
import urllib
import os
import glob

In [6]:
for i in range(1,13):
    for j in ['fhv','green','yellow']
        if len(str(i)) == 1:
            urllib.request.urlretrieve('https://s3.amazonaws.com/nyc-tlc/trip+data/%j_tripdata_2018-0%s.csv'%(j,i),'transportation/NewYork/%j_tripdata_2018-0%s.csv'%(j,i))
        elif len(str(i)) == 2:
            urllib.request.urlretrieve('https://s3.amazonaws.com/nyc-tlc/trip+data/%j_tripdata_2018-%s.csv'%(j,i),'transportation/NewYork/%j_tripdata_2018-%s.csv'%(j,i))
        print(j,i)

1
2
3
4
5
6
7
8
9
10
11
12


### Writte different months taxi data into one csv file

In [44]:
df = pd.read_csv(glob.glob('transportation/NewYork/*.csv')[2],nrows=100) 
print(glob.glob('transportation/NewYork/*.csv')[1])
df.columns

transportation/NewYork/green_tripdata_2017-08.csv


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')

In [ ]:
df = pd.read_csv(glob.glob('transportation/NewYork/*.csv')[0])  
df['Pickup_DateTime'] = pd.to_datetime(df['Pickup_DateTime'])
df['Date'] = df['Pickup_DateTime'].dt.strftime('%m/%d/%Y')
df['PassengerAmount'] = 1
df['PickupLocationID'] = df['PUlocationID']
df['DropoffLocationID'] = df['DOlocationID']
df['flag'] = (glob.glob('transportation/NewYork/*.csv')[0]).split('/')[2].split('_')[0]
df = df[['Date','PickupLocationID','DropoffLocationID','PassengerAmount']]
df.to_csv('transportation/NewYork/compacted/NewYork_Taxi.csv',index=False)

for i in glob.glob('transportation/NewYork/*.csv')[1:]:
    print(i)
    df = pd.read_csv(i)
    if str(i).split('/')[2].split('_')[0] == 'yellow':
        df = df.dropna(subset=['tpep_pickup_datetime','PULocationID', 'DOLocationID'])
        #no zone name is numbered as 0 and 264, 265 refer to NA
        df = df[(df['PULocationID']!=0)&(df['DOLocationID']!=0)]
        df = df[(df['PULocationID']!=264)&(df['DOLocationID']!=264)]
        df = df[(df['PULocationID']!=265)&(df['DOLocationID']!=265)]
        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
        df['Date'] = df['tpep_pickup_datetime'].dt.strftime('%m/%d/%Y')
        df['PassengerAmount'] = df['passenger_count']
        df['PickupLocationID'] = df['PULocationID']
        df['DropoffLocationID'] = df['DOLocationID']
    elif str(i).split('/')[2].split('_')[0] == 'green':
        df = df.dropna(subset=['lpep_pickup_datetime','PULocationID', 'DOLocationID'])
        df = df[(df['PULocationID']!=0)&(df['DOLocationID']!=0)]
        df = df[(df['PULocationID']!=264)&(df['DOLocationID']!=264)]
        df = df[(df['PULocationID']!=265)&(df['DOLocationID']!=265)]
        df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
        df['Date'] = df['lpep_pickup_datetime'].dt.strftime('%m/%d/%Y')
        df['PassengerAmount'] = df['passenger_count']
        df['PickupLocationID'] = df['PULocationID']
        df['DropoffLocationID'] = df['DOLocationID']
    elif str(i).split('/')[2].split('_')[0] == 'fhv':
        df = df.dropna(subset=['Pickup_DateTime','PUlocationID', 'DOlocationID'])
        df = df[(df['PUlocationID']!=0)&(df['DOlocationID']!=0)]
        df = df[(df['PUlocationID']!=264)&(df['DOlocationID']!=264)]
        df = df[(df['PUlocationID']!=265)&(df['DOlocationID']!=265)]
        df['Pickup_DateTime'] = pd.to_datetime(df['Pickup_DateTime'])
        df['Date'] = df['Pickup_DateTime'].dt.strftime('%m/%d/%Y')
        df['PassengerAmount'] = 1
        df['PickupLocationID'] = df['PUlocationID']
        df['DropoffLocationID'] = df['DOlocationID']
    df['flag'] = str(i).split('/')[2].split('_')[0]
    df = df[['Date','PickupLocationID','DropoffLocationID','PassengerAmount']]
    df.to_csv('transportation/NewYork/compacted/NewYork_Taxi.csv',index=False, header=False, mode='a+')